In [228]:
import csv
MAX_VERSES = 199
NUM_ANNOTATORS = 4

In [229]:
# Checks if the tsv file is valid and contents can be printed out
def printrows(tsv_file):
    annotations = open(tsv_file)
    read_tsv = csv.reader(annotations, delimiter="\t")
    row_count = 1
    for row in read_tsv:
        if row_count == MAX_VERSES:
            print("BREAK")
            break
        print(row)
        row_count += 1
        
printrows("Copy of ENGESV_Romans_segments - English1.tsv")

['B06___01_Romans______ENGESVN1DA_1_1240_13160', 'Paul, a servant  of Christ Jesus, called to be an apostle, set apart for the gospel of God,', '', 'X', '', '', 'NU', 'audio includes  Romans 1 ']
['B06___01_Romans______ENGESVN1DA_2_13160_17960', 'which he promised beforehand through his prophets in the holy Scriptures,', '', 'X', '', '', 'NU', '']
['B06___01_Romans______ENGESVN1DA_3_17960_23000', 'concerning his Son, who was descended from David  according to the flesh', '', 'X', '', '', 'NU', '']
['B06___01_Romans______ENGESVN1DA_4_23000_34280', 'and was declared to be the Son of God in power according to the Spirit of holiness by his resurrection from the dead, Jesus Christ our Lord,', 'X', '', '', '', 'P', '']
['B06___01_Romans______ENGESVN1DA_5_34280_45000', 'through whom we have received grace and apostleship to bring about the obedience of faith for the sake of his name among all the nations,', 'X', '', '', '', 'P', '']
['B06___01_Romans______ENGESVN1DA_6_45000_48960', 'including

In [230]:
# Raw numbers of total count of each sentiment for text only and speech only
def get_raw_numbers(tsv_file):
    annotations = open(tsv_file)
    read_tsv = csv.reader(annotations, delimiter="\t")
    # text only sentiment counters
    positive_count = 0
    negative_count = 0
    neutral_count = 0
    unsure_count = 0
    
    # speech only sentiment counters
    sp_positive = 0
    sp_negative = 0
    sp_neutral = 0
    sp_mixed = 0
    sp_unsure = 0
    
    row_count = 1
    for row in read_tsv:
        if row_count == MAX_VERSES:
            break
        row_count += 1
        if row[5] == '?' or row[5] == 'X':
            unsure_count += 1
        elif row[2].upper() == 'X':
            positive_count += 1
        elif row[3].upper() == 'X':
            neutral_count += 1
        elif row[4].upper() == 'X':
            negative_count += 1
        else:
            unsure_count += 1
        if row[6].upper() == 'P':
            sp_positive += 1
        elif row[6].upper() == 'N' or row[6].upper() == 'NG':
            sp_negative += 1
        elif row[6].upper() == 'NU':
            sp_neutral += 1
        elif row[6].upper() == 'M':
            sp_mixed += 1
        else:
            sp_unsure += 1
    annotations.close()
    
    print(tsv_file)
    print("pos:", positive_count)
    print("neg:", negative_count)
    print("neu:", neutral_count)
    print("unsure:", unsure_count)
    print("speech pos: ", sp_positive)
    print("speech neg: ", sp_negative)
    print("speech neu: ", sp_neutral)
    print("speech unsure: ", sp_unsure)
    print("speech mixed: ", sp_mixed)
        

In [231]:
get_raw_numbers("Copy of ENGESV_Romans_segments - English1.tsv")
get_raw_numbers("Copy of ENGESV_Romans_segments - English2.tsv")
get_raw_numbers("Copy of ENGESV_Romans_segments - chinese1.tsv")
get_raw_numbers("Copy of ENGESV_Romans_segments - chinese2_run.tsv")
get_raw_numbers("Copy of ENGESV_Romans_segments - chinese3_Lin.tsv")

Copy of ENGESV_Romans_segments - English1.tsv
pos: 98
neg: 65
neu: 18
unsure: 17
speech pos:  95
speech neg:  59
speech neu:  19
speech unsure:  6
speech mixed:  19
Copy of ENGESV_Romans_segments - English2.tsv
pos: 99
neg: 44
neu: 36
unsure: 19
speech pos:  84
speech neg:  39
speech neu:  44
speech unsure:  3
speech mixed:  28
Copy of ENGESV_Romans_segments - chinese1.tsv
pos: 20
neg: 25
neu: 16
unsure: 137
speech pos:  0
speech neg:  1
speech neu:  1
speech unsure:  195
speech mixed:  1
Copy of ENGESV_Romans_segments - chinese2_run.tsv
pos: 42
neg: 51
neu: 69
unsure: 36
speech pos:  38
speech neg:  49
speech neu:  68
speech unsure:  37
speech mixed:  6
Copy of ENGESV_Romans_segments - chinese3_Lin.tsv
pos: 92
neg: 54
neu: 52
unsure: 0
speech pos:  53
speech neg:  37
speech neu:  90
speech unsure:  3
speech mixed:  15


In [232]:
# Transfer the contents of a tsv file to a list for easier access
def tsv_to_list(annotation_file, max_verses):
    annotation_list = [] # rows are verses, columns are title, text, labels
    annotation = open(annotation_file)
    read_tsv = csv.reader(annotation, delimiter="\t") 
    verse_count = 1
    for row in read_tsv:
        if verse_count == max_verses:
            break
        annotation_list.append(row)
        verse_count += 1
    return annotation_list

In [233]:
# Get the number of annotators who marked positive, neutral, negative, or mixed for a given verse
def get_labels_for(verse, annotation_lists):
    labels = []
    verse_count = 0
    positive_count = 0
    neutral_count = 0
    negative_count = 0
    mixed_count = 0
    for annotation in annotation_lists:
        for row in annotation:
            verse_count += 1
            if verse_count != verse:
                continue
            useThisLabel = True
            if row[5] == '?' or row[5] == 'X': # don't use unsure
                useThisLabel = False
            if row[2].upper() == 'X' and (row[3].upper() != 'X' and row[4].upper() == 'X'): # don't use pos + neg mixed labels
                useThisLabel = False 
            elif row[2].upper() == 'X' and (row[3].upper() != 'X' and row[4].upper() != 'X'): # only pos
                positive_count += 1
            elif row[3].upper() == 'X' and (row[2].upper() != 'X' and row[4].upper() != 'X'): # only neu 
                neutral_count += 1
            elif row[4].upper() == 'X' and (row[3].upper() != 'X' and row[2].upper() != 'X'): # only neg
                negative_count += 1
            else: mixed_count += 1
            '''else:
                if row[2].upper() == 'X' and row[4].upper() != 'X': # only pos or pos + neu
                    positive_count += 1
                if row[3].upper() == 'X' and (row[2].upper() != 'X' and row[4].upper() != 'X'): # only neu 
                    neutral_count += 1
                if row[4].upper() == 'X' and row[2].upper() != 'X': # only neg or neg + neu
                    negative_count += 1'''
                
            if not useThisLabel:
                return []
        verse_count = 0
    labels.append(positive_count)
    labels.append(neutral_count)
    labels.append(negative_count)
    labels.append(mixed_count)
    return labels

In [234]:
# Get the number of annotators who marked positive, neutral, negative, or mixed for a given verse
def get_labels_for_speech(verse, annotation_lists):
    labels = []
    verse_count = 0
    positive_count = 0
    neutral_count = 0
    negative_count = 0
    mixed_count = 0
    for annotation in annotation_lists:
        for row in annotation:
            verse_count += 1
            if verse_count != verse:
                continue
            useThisLabel = True
            if '?' in row[6]: # don't use unsure
                useThisLabel = False
            if row[6].upper() == 'M': # don't use pos + neg mixed labels
                mixed_count += 1 
            elif row[6].upper() == 'P':
                positive_count += 1
            elif row[6].upper() == 'NU':
                neutral_count += 1
            elif row[6].upper() == 'NG' or row[6].upper() == 'N':
                negative_count += 1
            else: useThisLabel = False                
            if not useThisLabel:
                return []
        verse_count = 0
    labels.append(positive_count)
    labels.append(neutral_count)
    labels.append(negative_count)
    labels.append(mixed_count)
    return labels

In [235]:
# Get the labels for all of the verses and store in a 2D list that can be passed to the fleiss_kappa method
def get_fleiss_table(max_verses):
    annotation_list = ["Copy of ENGESV_Romans_segments - English1.tsv", "Copy of ENGESV_Romans_segments - English2.tsv", "Copy of ENGESV_Romans_segments - chinese1.tsv", "Copy of ENGESV_Romans_segments - chinese2_run.tsv", "Copy of ENGESV_Romans_segments - chinese3_Lin.tsv"]
    english1 = tsv_to_list(annotation_list[0], max_verses)
    english2 = tsv_to_list(annotation_list[1], max_verses)
    chinese1 = tsv_to_list(annotation_list[2], max_verses)
    chinese2 = tsv_to_list(annotation_list[3], max_verses)
    chinese3 = tsv_to_list(annotation_list[4], max_verses)
    annotations = []
    annotations.append(english1)
    annotations.append(english2)
    #annotations.append(chinese1)
    annotations.append(chinese2)
    annotations.append(chinese3)
    table = []
    for i in range(1, max_verses):
        labels = get_labels_for(i, annotations)
        if labels != []: # check if empty
            table.append(labels)
    return table
           

In [236]:
# Get the labels for all of the verses and store in a 2D list that can be passed to the fleiss_kappa method
def get_speech_fleiss_table(max_verses):
    annotation_list = ["Copy of ENGESV_Romans_segments - English1.tsv", "Copy of ENGESV_Romans_segments - English2.tsv", "Copy of ENGESV_Romans_segments - chinese1.tsv", "Copy of ENGESV_Romans_segments - chinese2_run.tsv", "Copy of ENGESV_Romans_segments - chinese3_Lin.tsv"]
    english1 = tsv_to_list(annotation_list[0], max_verses)
    english2 = tsv_to_list(annotation_list[1], max_verses)
    chinese1 = tsv_to_list(annotation_list[2], max_verses)
    chinese2 = tsv_to_list(annotation_list[3], max_verses)
    chinese3 = tsv_to_list(annotation_list[4], max_verses)
    annotations = []
    annotations.append(english1)
    annotations.append(english2)
    #annotations.append(chinese1)
    annotations.append(chinese2)
    annotations.append(chinese3)
    table = []
    for i in range(1, max_verses):
        labels = get_labels_for_speech(i, annotations)
        if labels != []: # check if empty
            table.append(labels)
    return table
           

In [237]:
fleiss_labels = get_fleiss_table(MAX_VERSES)
fleiss_labels_speech = get_speech_fleiss_table(MAX_VERSES)
#for i in range(len(fleiss_labels)):
#    print(i+1, end=' ')
#    print(fleiss_labels[i])
#    assert sum(fleiss_labels[i]) == 4
print(len(fleiss_labels))
print(fleiss_labels)
print(len(fleiss_labels_speech))
print(fleiss_labels_speech)

124
[[0, 4, 0, 0], [0, 4, 0, 0], [0, 4, 0, 0], [3, 1, 0, 0], [3, 1, 0, 0], [2, 2, 0, 0], [4, 0, 0, 0], [4, 0, 0, 0], [4, 0, 0, 0], [4, 0, 0, 0], [4, 0, 0, 0], [0, 2, 2, 0], [2, 2, 0, 0], [4, 0, 0, 0], [4, 0, 0, 0], [0, 0, 4, 0], [0, 4, 0, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [1, 1, 2, 0], [0, 0, 4, 0], [1, 0, 3, 0], [0, 1, 3, 0], [1, 3, 0, 0], [4, 0, 0, 0], [0, 0, 4, 0], [0, 0, 4, 0], [4, 0, 0, 0], [3, 1, 0, 0], [2, 2, 0, 0], [3, 1, 0, 0], [0, 4, 0, 0], [2, 2, 0, 0], [3, 1, 0, 0], [4, 0, 0, 0], [0, 0, 3, 1], [0, 0, 3, 1], [0, 0, 4, 0], [0, 0, 4, 0], [2, 1, 1, 0], [0, 0, 3, 1], [0, 3, 1, 0], [3, 1, 0, 0], [0, 3, 1, 0], [3, 1, 0, 0], [0, 0, 2, 2], [0, 3, 1, 0], [0, 0, 4, 0], [0, 1, 3, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [0, 0, 4, 0], [1, 1, 2, 0], [2, 2, 0, 0], [3, 1, 0, 0], [4, 0, 0, 0], [3, 1, 0, 0], [0, 4, 0, 0], [

In [238]:
def get_disagreed_verses(fleiss_table, max_annotators):
    disagreed_count = 0
    for i in range(len(fleiss_table)):
        if (fleiss_table[i] != [max_annotators, 0, 0, 0] and fleiss_table[i] != [0, max_annotators, 0, 0]) and (fleiss_table[i] != [0, 0, max_annotators, 0] and fleiss_table[i] != [0, 0, 0, max_annotators]):
            disagreed_count += 1
            print(i+1, end=': ')
            print(fleiss_table[i])
    print("Disagreed on", disagreed_count)

In [239]:
!pip install statsmodels
from statsmodels import stats
from statsmodels.stats import inter_rater

get_disagreed_verses(fleiss_labels, NUM_ANNOTATORS)
kappa = inter_rater.fleiss_kappa(fleiss_labels, method='fleiss')
print(kappa)

get_disagreed_verses(fleiss_labels_speech, NUM_ANNOTATORS)
speech_kappa = inter_rater.fleiss_kappa(fleiss_labels_speech, method='fleiss')
print(speech_kappa)

4: [3, 1, 0, 0]
5: [3, 1, 0, 0]
6: [2, 2, 0, 0]
12: [0, 2, 2, 0]
13: [2, 2, 0, 0]
30: [1, 1, 2, 0]
32: [1, 0, 3, 0]
33: [0, 1, 3, 0]
34: [1, 3, 0, 0]
39: [3, 1, 0, 0]
40: [2, 2, 0, 0]
41: [3, 1, 0, 0]
43: [2, 2, 0, 0]
44: [3, 1, 0, 0]
46: [0, 0, 3, 1]
47: [0, 0, 3, 1]
50: [2, 1, 1, 0]
51: [0, 0, 3, 1]
52: [0, 3, 1, 0]
53: [3, 1, 0, 0]
54: [0, 3, 1, 0]
55: [3, 1, 0, 0]
56: [0, 0, 2, 2]
57: [0, 3, 1, 0]
59: [0, 1, 3, 0]
66: [1, 1, 2, 0]
67: [2, 2, 0, 0]
68: [3, 1, 0, 0]
70: [3, 1, 0, 0]
72: [3, 0, 0, 1]
74: [1, 3, 0, 0]
75: [2, 1, 0, 1]
76: [2, 2, 0, 0]
77: [2, 2, 0, 0]
78: [3, 1, 0, 0]
80: [2, 2, 0, 0]
81: [0, 3, 0, 1]
82: [2, 2, 0, 0]
83: [2, 1, 0, 1]
84: [2, 2, 0, 0]
87: [3, 0, 0, 1]
89: [2, 2, 0, 0]
91: [3, 1, 0, 0]
92: [3, 1, 0, 0]
94: [3, 1, 0, 0]
95: [0, 1, 3, 0]
96: [0, 2, 2, 0]
97: [3, 1, 0, 0]
98: [3, 1, 0, 0]
102: [1, 2, 1, 0]
105: [3, 1, 0, 0]
106: [3, 1, 0, 0]
108: [1, 3, 0, 0]
109: [2, 1, 0, 1]
114: [3, 1, 0, 0]
115: [0, 1, 3, 0]
116: [2, 2, 0, 0]
120: [3, 1, 0, 0]
123: [2,

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.2b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### Text Data
| Desc | # English | # Chinese | # Verses used | Kappa | Disagreed verses | # Verses used (after mixed discarded)|
|------|----|--|--|--|--|--|
| all mixed labels counted under one category | 1&2 | 1&2&3 | 62 | 0.693 | na | na |
| pos + neg labels deleted, neutral mixed counted as a vote for either pos/neg | 1&2 | 1&2&3 | 62 | 0.748 | 17 | 53 |
| pos + neg labels deleted, neutral mixed counted as a vote for either pos/neg | 1&2 | 0 | 198 | 0.719 | 25 | 147 |
| pos + neg labels deleted, neutral mixed counted as a vote for either pos/neg | 0 | 1&2&3 | 62 | 0.777 | 12 | 56 |
| pos + neg labels deleted, neutral mixed counted as a vote for either pos/neg | 0 | 2&3 | 198 | 0.531 | 49 | 157 |
| pos + neg labels deleted, neutral mixed counted as a vote for either pos/neg | 1&2 | 2&3 | 198 | 0.624 | 53 | 124 |
| pos + neg labels deleted, other mixed counted under one category | 1&2 | 2&3 | 198 | 0.582 | 59 | 124 |
| pos + neg labels deleted, other mixed counted under one category | 1&2 | 1&2&3 | 62 | 0.707 | 20 | 53 |
| pos + neg labels deleted, other mixed counted under one category | 1&2 | 0 | 198 | 0.635 | 34 | 147 |
| pos + neg labels deleted, other mixed counted under one category | 0 | 1&2&3 | 62 | 0.747 | 14 | 56 |
| pos + neg labels deleted, other mixed counted under one category | 0 | 2&3 | 198 | 0.502 | 53 | 157 |

### Speech Data
| Desc | # English | # Chinese | # Verses used | Kappa | Disagreed verses | # Verses used (after mixed discarded)|
|------|----|--|--|--|--|--|
| all mixed labels counted under one category | 1&2 | 2&3 | 62 | 0.406 | 30 | 50 |
| all mixed labels counted under one category | 1&2 | 2&3 | 198 | 0.406 | 30 | 50 |